<a href="https://colab.research.google.com/github/pinkdolphin11/ESAA/blob/main/%EC%95%99%EC%83%81%EB%B8%94_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **| 앙상블 학습과 랜덤 포레스트 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch07 앙상블 학습과 랜덤 포레스트 연습문제 2, 7, 8, 9번
- 이론적 지식을 묻는 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

In [1]:
# import libraries
import numpy as np

### **1. 직접 투표와 간접 투표 분류기 사이의 차이점은 무엇일까요?**
___


직접 투표 분류기는 각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 다수결 투표로 정하는 분류기이다. 반면 간접 투표 분류기는 개별 분류기 예측들의 평균을 구해서 확률이 가장 높은 클래스를 예측하는 것이다.

### **2. 그레디언트 부스팅 앙상블이 훈련 데이터에 과대 적합되었다면 학습률을 어떻게 해야 할까요?**
___

학습률을 높여야 한다.

### **3. [실습] 다음 지시에 따라 투표 기반 분류 모델을 만들어 보세요**
___

#### **STEP 1. MNIST 데이터를 불러들이고, 훈련, 검증, 테스트 데이터로 나누세요.**

In [2]:
# import MNIST dataset
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame = False)
X, y = mnist["data"], mnist["target"]

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_rem, y_train, y_rem = train_test_split(X,y)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem)

####  **STEP 2. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시키세요.**
- 모델 파라미터는 `n_estimators=100`, `random_state=42`로 설정합니다. 

In [4]:
# import package
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [5]:
# model fitting - RandomForest

rf_clf = RandomForestClassifier(n_estimators=100,random_state=42)
ex_clf = ExtraTreesClassifier(n_estimators=100,random_state=42)
lsvc_clf = LinearSVC(random_state=42)
mlp_clf = MLPClassifier(random_state=42)

####  **STEP 3-1. 앞에서 훈련시킨 각 모델을 직접 투표 방법을 사용해 앙상블로 연결하고 훈련시킨 후, `score()`메서드를 이용하여 검증 데이터셋에서의 성능을 평가해보세요.**

In [6]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(estimators=[('rf',rf_clf),('ex',ex_clf),('lsvc',lsvc_clf),('mlp',mlp_clf)],voting='hard')

In [7]:
# model fitting

voting_clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('ex', ExtraTreesClassifier(random_state=42)),
                             ('lsvc', LinearSVC(random_state=42)),
                             ('mlp', MLPClassifier(random_state=42))])

In [8]:
# model test

voting_clf.score(X_valid,y_valid)

0.9702857142857143

####  **STEP 3-2. 검증 데이터셋에서 각 분류 모델의 성능을 `score()` 메서드를 이용하여 확인해보고, 가장 성능이 낮은 모델을 제거하여 그 결과를 비교해보세요.**
- Hint : 가장 성능이 낮은 모델을 제거할 때 `set_params`를 활용해보세요

In [9]:
# 각 분류 모델 학습

rf_clf.fit(X_train,y_train)
ex_clf.fit(X_train,y_train)
lsvc_clf.fit(X_train,y_train)
mlp_clf.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


MLPClassifier(random_state=42)

In [10]:
# 각 분류 모델의 성능 확인

print(rf_clf.score(X_valid,y_valid))
print(ex_clf.score(X_valid,y_valid))
print(lsvc_clf.score(X_valid,y_valid))
print(mlp_clf.score(X_valid,y_valid))

0.9677714285714286
0.9705904761904762
0.8332190476190476
0.9588571428571429


- Q. 어떤 모델의 성능이 가장 낮나요?
- A. linear svc model

In [11]:
# 가장 성능이 낮은 모델 제거

del voting_clf.estimators_[2]

In [12]:
# model fitting
from sklearn.ensemble import VotingClassifier
voting_clf_new = VotingClassifier(estimators=[('rf', rf_clf), ('ex', ex_clf), ('mlp', mlp_clf)],voting='hard')
voting_clf_new.fit(X_train, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('ex', ExtraTreesClassifier(random_state=42)),
                             ('mlp', MLPClassifier(random_state=42))])

In [13]:
# 모델 제거 후 성능 확인
voting_clf_new.score(X_valid, y_valid)

0.9724190476190476

### **4. 다음 단계를 따라 앞에서 훈련시킨 분류 모델들을 이용하여 스태킹 앙상블을 구성해보자.**
___

#### **STEP 1. 3번 문제의 각 분류 모델을 실행해서 검증 세트에서 예측을 만들고, 그 결과로 훈련 세트를 만들어 보세요.**

In [15]:
y_pred_rf = rf_clf.predict(X_valid)
y_pred_ex = ex_clf.predict(X_valid)
y_pred_lsvc = lsvc_clf.predict(X_valid)
y_pred_mlp = mlp_clf.predict(X_valid)

predictions = np.column_stack([y_pred_rf, y_pred_ex, y_pred_lsvc, y_pred_mlp])

####  **STEP 2. 새로운 훈련 세트를 이용하여 랜덤 포레스트 분류 모델을 학습시켜 보세요.**

In [16]:
rf_clf.fit(predictions,y_valid)

RandomForestClassifier(random_state=42)

- 이 랜덤 포레스트 분류 모델이 바로 블렌더에 해당합니다. 

####  **STEP 3. 이제 테스트셋에서 스태킹 앙상블 모델을 평가해보세요.**
- 성능 평가 지표로 **정확도**를 이용하세요.

In [ ]:
# 각 분류 모델의 예측을 만들어 새로운 데이터셋 생성
rf_pred = rf_clf.predict(X_test)
ex_pred = ex_clf.predict(X_test)
lsvc_pred = lsvc_clf.predict(X_test)
mlp_pred = mlp_clf.predict(X_test)

preds = np.array([rf_pred, ex_pred, lsvc_pred, mlp_pred])
preds = np.transpose(preds)

In [ ]:
# 새로운 데이터셋을 이용하여 블렌더로 예측
y_pred = rf_clf.predict(preds)

In [ ]:
# model test
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)